<a href="https://colab.research.google.com/github/AnatoliyZhuk/Knowledge/blob/main/I_Profi_part_1/I_Profi_part_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>